In [14]:
import numpy as np
import scipy
from scipy import stats
import scipy.ndimage as ndimage
import astropy
from astropy.stats import sigma_clipped_stats
import math
import time
from photutils import DAOStarFinder
from photutils import aperture_photometry
from photutils import CircularAperture
from photutils import CircularAnnulus

#for reading in fits files
from astropy.io import fits
from astropy.wcs import WCS

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join, exists
%matplotlib inline                           
%config InlineBackend.figure_format='retina'      
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pylab as plt                     
plt.rcParams['figure.figsize'] = (14.0, 14.0)    
import pandas as pd

In [15]:
file = "/home/usuario/Documents/Mestrado/FFIs/testeall/clean/masters/kic_cut.csv"
kic2 = pd.read_csv(file,sep=',',header=[0])

kic_sem_mag = kic2[(kic2.kic_kepmag.isna()) & (kic2.kct_num_season_onCCD!=0)]
kic_com_mag = kic2[(kic2.kic_kepmag.notna()) & (kic2.kct_num_season_onCCD!=0)]
kic_visto = kic2[(kic2.kct_num_season_onCCD!=0)]

In [16]:
def kic_skygroup(vector,n_sky):
    new = vector[(vector.kct_sky_group_id_value==n_sky)]
    return new

In [17]:
def coord_min(x,y,xs,ys):
    array=[]
    dif=[]
    closest=[]
    sigmax = np.std(xs)
    sigmay = np.std(ys)
    for i in range(len(xs)):
        if abs(x - xs[i])<sigmax/50 and abs(y - ys[i])<sigmay/50:
            #array.append([xs[i],ys[i]])
            dif.append(np.sqrt((x - xs[i])**2+(y - ys[i])**2))
            if np.min(dif) == np.sqrt((x - xs[i])**2+(y - ys[i])**2):
                closest.append([xs[i],ys[i]])
    return closest[-1]

## Here I define which season

In [18]:
caldir = '/home/usuario/Documents/Mestrado/FFIs/testeall/clean/masters/season3/'
#get the positions from the star list if one is provided 
os.chdir(caldir) #changes to the raw image directory
files = [f for f in glob.glob("*.fits") if isfile(join(caldir, f))]
files.sort()

In [19]:
!pwd

/home/usuario/Documents/Mestrado/FFIs/testeall/clean/masters/season3


In [20]:
for i in range(16,len(files)+1):
    print(i)
    a = fits.open(files[i-1])
    mast = fits.getdata(caldir+files[i-1])
    w = WCS(a[1].header,'image')
    table = kic_skygroup(kic_visto,i)
    kicid = np.array(table.kic_kepler_id)
    kmag = np.array(table.kic_kepmag)
    ra = table.kic_degree_ra
    dec = table.kic_dec
    #print(len(ra),len(dec))
    x, y = w.all_world2pix(ra, dec,0,ra_dec_order=True,detect_divergence=True)
    positions = (x, y)

    rads = np.arange(2,5,.25) 

    #do the aperture photometry and find the optimal aperture
    apertures = [CircularAperture(positions, r=r) for r in rads]
    phot_table = aperture_photometry(mast, apertures, method = 'exact')
    idx = 0

    offset = np.zeros((len(rads),len(x)))
    for ii in range(0, len(x)):
        if (x[ii] > 3) and (x[ii] < 1097) and (y[ii] > 3) and (y[ii] < 1021):
            dist = np.sqrt((x[ii]-x)**2+(y[ii]-y)**2)
            chk = np.where(dist < 6.)
            if (len(chk[0]) == 1):
                for jj in range(1, len(rads)):
                    mg1 = 25.-2.5*np.log10(phot_table[ii][jj+3])
                    mg0 = 25.-2.5*np.log10(phot_table[ii][jj+2])
                    offset[jj,ii] = mg1-mg0
    

    prv = 1.
    opt_rad = 10.
    for ii in range(0, len(rads)):
        chk = np.median(offset[ii,:])

        if (np.abs(chk-prv) <= 0.001) and (rads[ii] < opt_rad):	
            opt_rad = rads[ii]
            #print('The optimal aperture size is '+str(opt_rad)+'.')
        if (np.abs(chk-prv) > 0.001):
            prv = chk

    #do the aperture photometry
    apertures = CircularAperture(positions, r = opt_rad)
    phot_table = aperture_photometry(mast, apertures, method = 'exact')

    #get the background of the image
    cimg, clow, chigh = scipy.stats.sigmaclip(mast, low=2.5, high = 2.5) #do a 2.5 sigma clipping
    bkg_mean = np.median(cimg) #determine the sky value
    sig = np.std(cimg) #determine the sigma(sky)

    #convert to magnitudes
    #aqui nao estou retirando background das imagens de ref
    flx = phot_table['aperture_sum']#-(bkg_mean*(np.pi*opt_rad**2))
    flx_er = np.sqrt(phot_table['aperture_sum'])
    x_pix = x
    y_pix = y

    #create the magnitudes from the flux
    mag = 25.0-2.5*np.log10(flx)
    err = (2.5/np.log(10.))*(flx_er/flx)

    #write the magnitudes to a file
    output = open(caldir+str(i)+'_master.ap', 'w')
    for ii in range(0,len(phot_table['id'])):
        if (x_pix[ii] > 3) and (x_pix[ii] < 1097) and (y_pix[ii] > 3) and (y_pix[ii] < 1021) and (np.isnan(mag[ii]) == False):
            output.write(str(kicid[ii])+','+str(x_pix[ii])+','+str(y_pix[ii])+','+str(kmag[ii])+','+str(mag[ii])+','+str(err[ii])+'\n')
    output.close()

    #write the fluxes to a file
    output = open(caldir+str(i)+'_master.flux', 'w')
    for ii in range(0, len(phot_table['id'])):
        if (x_pix[ii] > 3) and (x_pix[ii] < 1097) and (y_pix[ii] > 3) and (y_pix[ii] < 1021) and (np.isnan(mag[ii]) == False):
            output.write(str(kicid[ii])+','+str(x_pix[ii])+','+str(y_pix[ii])+','+str(flx[ii])+','+str(flx_er[ii])+'\n')
    output.close()

    #write the star list to a file
    output = open(caldir+str(i)+'_starlist.txt', 'w')
    for ii in range(0, len(phot_table['id'])):
        if (x_pix[ii] > 3) and (x_pix[ii] < 1097) and (y_pix[ii] > 3) and (y_pix[ii] < 1021) and (np.isnan(mag[ii]) == False):
            output.write(str(kicid[ii])+','+str(x_pix[ii])+','+str(y_pix[ii])+'\n')
    output.close()

16
17
18
19
20


/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in sqrt
/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in log10


21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log10
/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in log10
/home/usuario/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


50
51
52
53
54
55
56
57
58
59


/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log10
/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide
/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide


60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


## After you have to take out the module_3 images and tables of each season, which are:

### 77,78,79,80 for season 0
### 33,34,35,36 for season 1
### 5,6,7,8         for season 2
### 49,50,51,52 for season 3